In [3]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
from plotly.subplots import make_subplots

import pandas as pd

import pymysql

In [221]:
信義_df = pd.read_csv(r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/信義全資料/信義全資料.csv')
永慶_df = pd.read_csv(r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/2022_6.csv')
人口資料 = pd.read_csv(r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/平均收入.csv')

# 視覺化

In [176]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )
cursor = db.cursor()

command = """select * from NEWEST_INFO where Type in ('公寓')"""


cursor.execute(command)

data_住宅大樓_永慶 = cursor.fetchall()
db.commit()
db.close()

In [20]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )
cursor = db.cursor()

command = """select * from HSINYI_INFO where Type in ('大樓','華廈')"""


cursor.execute(command)

data_住宅大樓_信義 = cursor.fetchall()
db.commit()
db.close()

In [171]:
data_住宅大樓_信義_df = pd.DataFrame(data_住宅大樓_信義)
data_住宅大樓_信義_df = data_住宅大樓_信義_df[data_住宅大樓_信義_df[7]!='未提供']
data_住宅大樓_信義_df[7] = data_住宅大樓_信義_df[7].astype('float')

In [197]:
data_住宅大樓_永慶_df = pd.DataFrame(data_住宅大樓_永慶)
data_住宅大樓_永慶_df = data_住宅大樓_永慶_df[data_住宅大樓_永慶_df[9]!='未提供']
data_住宅大樓_永慶_df[9] = data_住宅大樓_永慶_df[9].astype('float')

In [198]:
data_住宅大樓_永慶_df_各區平均 = data_住宅大樓_永慶_df[[3,9,14]].groupby([14]).mean()
data_住宅大樓_信義_df_各區平均 = data_住宅大樓_信義_df[[3,7,13]].groupby([13]).mean()

In [199]:
data_住宅大樓_永慶_df_各區平均

,3,9
14,,
中山區,5606.772595,55.498659
中正區,5810.697531,55.701667
信義區,6112.805970,54.650746
內湖區,5314.277778,68.419676
北投區,3997.892473,60.088925
南港區,5390.825243,62.408252
士林區,5250.988636,64.369773
大同區,3564.100000,51.571571
大安區,6470.924479,54.107995


In [200]:
data_住宅大樓_永慶_df_各區計數 = data_住宅大樓_永慶_df[[0,14]].groupby([14]).count()
data_住宅大樓_信義_df_各區計數 = data_住宅大樓_信義_df[[0,13]].groupby([13]).count()
data_住宅大樓_df_各區計數 = data_住宅大樓_永慶_df_各區計數.merge(data_住宅大樓_信義_df_各區計數,right_index=True, left_index=True)

In [201]:
人口資料_109 = 人口資料[(人口資料['"年別']=='"109年') & (人口資料['行政區']!='總平均')].iloc[:,[1,3,4]]

In [202]:
人口資料_109 = 人口資料_109.set_index('行政區').loc[data_住宅大樓_df_各區計數.index]

In [205]:
fig = make_subplots(rows=4, cols=1,row_heights = [0.15,0.4,0.2,0.25],subplot_titles = ('BoxPlot','均價及人均所得比較','戶數','建坪'),
                    specs = [[{"secondary_y": False}],
                             [{"secondary_y": True}],
                             [{"secondary_y": False}],
                             [{"secondary_y": False}]],
                    shared_xaxes='rows')

fig.add_trace(go.Box(x=data_住宅大樓_永慶_df[3],name='永慶',fillcolor='#F1C232',marker_color='#F1C232'),row=1,col=1)
fig.add_trace(go.Box(x=data_住宅大樓_信義_df[3],name='信義',fillcolor='#538e3a',marker_color='#538e3a'),row=1,col=1)



fig.add_trace(go.Bar(x=data_住宅大樓_永慶_df_各區平均.index,y=data_住宅大樓_永慶_df_各區平均[3],name='永慶',marker_color='#F1C232'),row=2,col=1)
fig.add_trace(go.Bar(x=data_住宅大樓_信義_df_各區平均.index,y=data_住宅大樓_信義_df_各區平均[3],name='信義',marker_color='#538e3a'),row=2,col=1)
fig.add_trace(go.Scatter(x=人口資料_109.index,y=人口資料_109['[一]所得收入總計[元]'],name='所得收入總計',marker_color='#1f496e'),row=2,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=人口資料_109.index,y=人口資料_109['[三]可支配所得[元]"'],name='可支配所得',marker_color='#71244c'),row=2,col=1,secondary_y=True)


fig.add_trace(go.Bar(x=data_住宅大樓_df_各區計數.index,y=data_住宅大樓_df_各區計數['0_x'],name='永慶',marker_color='#F1C232',offsetgroup=0,base= data_住宅大樓_df_各區計數['0_y'],text=data_住宅大樓_df_各區計數['0_x']),row=3,col=1)
fig.add_trace(go.Bar(x=data_住宅大樓_df_各區計數.index,y=data_住宅大樓_df_各區計數['0_y'],name='信義',marker_color='#538e3a',offsetgroup=0),row=3,col=1)


fig.add_trace(go.Scatter(x=data_住宅大樓_永慶_df_各區平均.index,y=data_住宅大樓_永慶_df_各區平均[9],name='永慶',marker_color='#F1C232'),row=4,col=1)
fig.add_trace(go.Scatter(x=data_住宅大樓_信義_df_各區平均.index,y=data_住宅大樓_信義_df_各區平均[7],name='信義',marker_color='#538e3a'),row=4,col=1)



fig.update_layout(title=dict(
    font = dict(
        color = 'blue',
        size  = 23
    ),
    text = '大樓類型建物比較'
))

fig.for_each_xaxis(lambda x: x.update(showgrid=False))
fig.for_each_yaxis(lambda x: x.update(showgrid=False))

fig.show()


# 視覺化模組

In [277]:
import re
list_信義 = []
for x in 信義_df['Type'].unique():
    if re.match('套房',x) and '辦公' not in x:
        list_信義.append(x)

list_永慶 = []
for x in 永慶_df['Type'].unique():
    if re.match('套房',x):
        list_永慶.append(x)

In [275]:
def make_plot(信義,永慶,kind):    
    def get_data(信義,永慶):
        if len(信義) == 1:
            信義.append('透')
        if len(永慶) == 1:
            永慶.append('透')


            
        db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )
        cursor = db.cursor()

        var  = 信義
        command = f"""SELECT * FROM HSINYI_INFO WHERE `Type` IN {repr(tuple(map(str, var)))}"""


        cursor.execute(command)

        data_信義 = cursor.fetchall()
        db.commit()
        db.close()

        db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )
        cursor = db.cursor()

        var2  = 永慶
        command2 = f"""SELECT * FROM NEWEST_INFO WHERE `Type` IN {repr(tuple(map(str, var2)))}"""


        cursor.execute(command2)

        data_永慶 = cursor.fetchall()
        db.commit()
        db.close()

        return pd.DataFrame(data_信義),  pd.DataFrame(data_永慶)
        
    信義_df, 永慶_df = get_data(信義,永慶)
    人口統計         = pd.read_csv(r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/平均收入.csv')
    信義_df    = 信義_df[信義_df[7]!='未提供']
    信義_df[7] = 信義_df[7].astype('float')
    永慶_df    = 永慶_df[永慶_df[9]!='未提供']
    永慶_df[9] = 永慶_df[9].astype('float')


    data_永慶_df_各區平均 = 永慶_df[[3,9,14]].groupby([14]).mean()
    data_信義_df_各區平均 = 信義_df[[3,7,13]].groupby([13]).mean()
    data_永慶_df_各區計數 = 永慶_df[[0,14]].groupby([14]).count()
    data_信義_df_各區計數 = 信義_df[[0,13]].groupby([13]).count()
    #####
    data_df_各區計數 = data_永慶_df_各區計數.merge(data_信義_df_各區計數,right_index=True, left_index=True,how='outer')
    人口統計_109 = 人口統計[(人口統計['"年別']=='"109年') & (人口統計['行政區']!='總平均')].iloc[:,[1,3,4]]
    人口統計_109 = 人口統計_109.set_index('行政區').loc[data_df_各區計數.index]



    fig = make_subplots(rows=4, cols=1,row_heights = [0.15,0.4,0.2,0.25],subplot_titles = ('BoxPlot','均價及人均所得比較','戶數','建坪'),
                    specs = [[{"secondary_y": False}],
                             [{"secondary_y": True}],
                             [{"secondary_y": False}],
                             [{"secondary_y": False}]])

    fig.add_trace(go.Box(x=永慶_df[3],name='永慶',fillcolor='#F1C232',marker_color='#F1C232'),row=1,col=1)
    fig.add_trace(go.Box(x=信義_df[3],name='信義',fillcolor='#538e3a',marker_color='#538e3a'),row=1,col=1)



    fig.add_trace(go.Bar(x=data_永慶_df_各區平均.index,y=data_永慶_df_各區平均[3],name='永慶',marker_color='#F1C232'),row=2,col=1)
    fig.add_trace(go.Bar(x=data_信義_df_各區平均.index,y=data_信義_df_各區平均[3],name='信義',marker_color='#538e3a'),row=2,col=1)
    fig.add_trace(go.Scatter(x=人口統計_109.index,y=人口統計_109['[一]所得收入總計[元]'],name='所得收入總計',marker_color='#1f496e'),row=2,col=1,secondary_y=True)
    fig.add_trace(go.Scatter(x=人口統計_109.index,y=人口統計_109['[三]可支配所得[元]"'],name='可支配所得',marker_color='#71244c'),row=2,col=1,secondary_y=True)

    fig.add_trace(go.Bar(x=data_df_各區計數.index,y=data_df_各區計數['0_x'],name='永慶',marker_color='#F1C232',offsetgroup=0,base= data_df_各區計數['0_y'],text=data_df_各區計數['0_x']),row=3,col=1)
    fig.add_trace(go.Bar(x=data_df_各區計數.index,y=data_df_各區計數['0_y'],name='信義',marker_color='#538e3a',offsetgroup=0),row=3,col=1)

    fig.add_trace(go.Scatter(x=data_永慶_df_各區平均.index,y=data_永慶_df_各區平均[9],name='永慶',marker_color='#F1C232'),row=4,col=1)
    fig.add_trace(go.Scatter(x=data_信義_df_各區平均.index,y=data_信義_df_各區平均[7],name='信義',marker_color='#538e3a'),row=4,col=1)




    fig.update_layout(title=dict(
        font = dict(
            color = 'blue',
            size  = 23
        ),
        text = kind + ' Plot'
    ))

    fig.for_each_xaxis(lambda x: x.update(showgrid=False))
    fig.for_each_yaxis(lambda x: x.update(showgrid=False))

    fig.show()
    

In [278]:
make_plot(信義=list_信義,
            永慶=list_永慶,kind='套房')